In [ ]:
from conflux_web3.dev import get_testnet_web3
from cns import CNS
from conflux_web3.types import Drip
from ens.utils import raw_name_to_hash
from hexbytes import HexBytes
from typing import get_type_hints
from uuid import uuid4
import time
import os



In [ ]:
# initialize variables
w3 = get_testnet_web3()
sk = os.environ.get("TESTNET_SECRET", None)
if sk:
    account = w3.account.from_key(sk)
    w3.cfx.default_account = account
else:
    account = w3.account.create()
    w3.cfx.default_account = account
    faucet = w3.cfx.contract(name="Faucet")
    faucet.functions.claimCfx().transact().executed()

In [ ]:
ens_registry = w3.cns.ens
controller = w3.cfx.contract(address="cfxtest:acgeyhvevvs2y2ppgtkhckn4sghwe4n4se734ed7tn", name="ETHRegistrarController")
wrapper = w3.cfx.contract("cfxtest:acd72jvhy174x6rs8azsjfvbp7brfvm4fenn0yk9c4", name="NameWrapper")
resolver = w3.cfx.contract("cfxtest:acahw54guhthk9778mxxk5jkn1pjfzbuz6xs73hcb4", name="RESOLVER")

In [ ]:
# register a random domain using web3controller
ens_name = str(uuid4())
duration = 60 * 60 * 24 * 30 * 12
secret = w3.keccak(text="114514")
# resolver = w3.cfx.address.zero_address(1)
resolver_address = resolver.address
data = []
fuses = 0
wrapperExpiry = 2**64-1
reverse = False

commitment = controller.functions.makeCommitment(
    ens_name, str(account.address), duration, secret, resolver_address, data, reverse, fuses, wrapperExpiry
).call()
price = Drip(controller.caller.rentPrice(ens_name, 60 * 60 * 24 * 30 * 12)[0]).to("CFX") + 20
receipt = controller.functions.commit(commitment).transact().executed()
time.sleep(130)
try:
    controller.functions.register(
        ens_name, str(account.address), duration, secret, resolver_address, data, reverse, fuses, wrapperExpiry
    ).transact({
        "value": price,
    }).executed()
    print(f"name {ens_name}.web3 is registered")
except Exception as e:
    print(e)

In [ ]:
registered_name = f"{ens_name}.web3"
print(f"raw owner: {w3.cns.owner(registered_name)}")
print(f"owner from name wrapper: {w3.cns.owner(registered_name, wrapped=True)}")

In [ ]:
# cns write apis are still unstable, it is recommended to interact with the contracts
print(f"Resolve {registered_name}: {w3.cns.address(registered_name)}")
resolver.functions.setAddr(raw_name_to_hash(registered_name), account.address).transact().executed()
print(f"Resolve {registered_name}: {w3.cns.address(registered_name)}")

random_subdomain = str(uuid4())
full_subdomain = f"{random_subdomain}.{registered_name}"
wrapper.functions.setSubnodeOwner(raw_name_to_hash(registered_name), random_subdomain, account.address, fuses, wrapperExpiry).transact().executed()
wrapper.functions.setResolver(raw_name_to_hash(full_subdomain), resolver.address).transact().executed()
resolver.functions.setAddr(raw_name_to_hash(full_subdomain), account.address).transact().executed()
print(f"Resolve {full_subdomain}: {w3.cns.address(full_subdomain)}")

In [ ]:
# or you can manually enable write apis
# only `setup_owner` and `setup_address` are available
w3.cns.allow_unstable_api = True
# tx_hash = w3.cns.setup_address(registered_name, account.address, wrapped=True)
# assert tx_hash is not None
# tx_hash.executed()
# assert w3.cns.address(registered_name) == account.address
# print(f"Resolve {registered_name}: {w3.cns.address(registered_name)}")

# set subdomain
random_subdomain = str(uuid4())
full_subdomain = f"{random_subdomain}.{registered_name}"
w3.cns.setup_address(full_subdomain, account.address, wrapped=True).executed()
assert w3.cns.address(full_subdomain) == account.address
print(f"Resolve {full_subdomain}: {w3.cns.address(full_subdomain)}")
